In [45]:
from src.pdfinsight import pdf_extractor
import pandas as pd

In [46]:
df = pdf_extractor("220 Performance Report (Construction).pdf", toc_pages = 1, )
df

,file,page,block,xmin,ymin,xmax,ymax,text,font_size,font_characteristics,...,image,text_rep,xmin_rep,ymin_rep,cat,xmin_round,ymin_round,block_ymin_diff,refined_block,list_block
0,220 Performance Report (Construction).pdf,1,3,204.119781,107.255196,442.108307,121.291252,SAFETY AND CONTRACTS GROUP,14.04,20,...,,1,1,1,toc,204.1,107.0,NaN,1,False
1,220 Performance Report (Construction).pdf,1,3,133.678894,149.989777,514.405884,171.943481,CONTRACT PROCEDURE MANUAL,21.95,20,...,,1,1,1,toc,133.7,150.0,NaN,1,False
2,220 Performance Report (Construction).pdf,1,3,259.439178,173.975174,383.152130,188.011215,(CONSTRUCTION),14.04,20,...,,1,1,1,toc,259.4,174.0,NaN,1,False
3,220 Performance Report (Construction).pdf,1,3,154.798599,259.722534,492.416595,277.717407,PREPARING PERFORMANCE REPORT,17.99,20,...,,1,1,1,toc,154.8,260.0,NaN,1,False
4,220 Performance Report (Construction).pdf,1,3,204.599152,344.802612,442.616943,362.797485,Document No.: CPM/C/M/220,17.99,20,...,,1,1,1,toc,204.6,345.0,NaN,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,220 Performance Report (Construction).pdf,6,8,125.279114,148.928619,453.813110,160.925079,contractor’s/consultant’s performance with the...,12.00,4,...,,1,70,2,content,125.3,149.0,30.0,73,True
258,220 Performance Report (Construction).pdf,6,8,458.279327,148.928619,464.277557,160.925079,.,12.00,4,...,,2,1,2,content,458.3,149.0,30.0,73,True
259,220 Performance Report (Construction).pdf,6,11,89.879158,763.199951,93.117302,769.678101,4,6.48,4,...,,2,11,2,footnote,89.9,763.0,614.0,74,False
260,220 Performance Report (Construction).pdf,6,11,93.119171,764.038208,555.447998,773.995178,BCA’s Advisory Note on ‘Review of Performance ...,9.96,4,...,,1,4,3,footnote,93.1,764.0,614.0,74,False


In [48]:
# we check the ymin difference between each row INSTEAD OF BLOCK. 
# this is important in determining the para_thres
# text below the para_thres would be considered to be in the same block of text (ie para) 
# seems like if those between 0 to 20 is above 70%, we can set that as para_thres
pd.DataFrame(pd.cut([idx for idx in df['ymin_round'].diff()], [-1000, -0.01,10, 20, 30, 40, 50, 1000])).value_counts(normalize=True).sort_index()

(-1000.0, -0.01]    0.019157
(-0.01, 10.0]       0.452107
(10.0, 20.0]        0.325670
(20.0, 30.0]        0.126437
(30.0, 40.0]        0.026820
(40.0, 50.0]        0.019157
(50.0, 1000.0]      0.030651
Name: proportion, dtype: float64

In [49]:
# double check the ymin_thres between REFINED_BLOCKS
# as long as our defined para_thres of 20 is the majority
# then should be safe to use that as para_thres
pd.DataFrame(pd.cut([idx for idx in df['block_ymin_diff']], [-1000, -0.01,10, 20, 30, 40, 50, 1000])).value_counts(normalize=True).sort_index()

(-1000.0, -0.01]    0.093385
(-0.01, 10.0]       0.000000
(10.0, 20.0]        0.431907
(20.0, 30.0]        0.276265
(30.0, 40.0]        0.112840
(40.0, 50.0]        0.027237
(50.0, 1000.0]      0.058366
Name: proportion, dtype: float64